In [739]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as op

In [740]:
def sigmoid(z):
    result = 1.0 / (1.0 + np.exp(-z))
    
    return result

In [741]:
def sigmoidGradient(z):
    result = np.multiply(sigmoid(z), (1-sigmoid(z)))
    
    return result

In [742]:
def debugInitializeWeights(fan_out, fan_in):
    W = np.zeros((fan_out, fan_in+1))
    W = np.reshape(W, (1, -1))
    
    for i in range(W.shape[1]):
        W[0][i] = np.sin(i+1) / 10
    
    W = np.reshape(W, (fan_out, fan_in+1))
    
    return W

In [ ]:
def computeNumericalGradient(J, theta):
    
    
    
    
    

In [743]:
def checkNNGradients(lambda_t):
    input_layer_size = 3
    hidden_layer_size = 5
    num_labels = 3
    m = 5
    
    Theta1 = debugInitializeWeights(hidden_layer_size, input_layer_size)
    Theta2 = debugInitializeWeights(num_labels, hidden_layer_size)
    X  = debugInitializeWeights(m, input_layer_size - 1)
    y = np.zeros((m, 1))
    for i in range(m):
        y[i][0] = 1 + (i+1) % num_labels
    y = y.astype(int)
    
    nn_params = np.concatenate((Theta1.flatten(), Theta2.flatten()))
    cost = computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
    grad = computeGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
    
    

In [744]:
def computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t):

    Theta1_flat = nn_params[ : hidden_layer_size * (input_layer_size + 1)]
    Theta2_flat = nn_params[hidden_layer_size * (input_layer_size + 1) : ]
    
    Theta1 = np.reshape(Theta1_flat, (hidden_layer_size, input_layer_size+1))
    Theta2 = np.reshape(Theta2_flat, (num_labels, hidden_layer_size+1))
    
    m = X.shape[0]

    a1 = np.c_[np.ones([m, 1]), X]    # Add a column of ones to x
    z2 = np.dot(a1, Theta1.T)    # 5000*25;

    a2 = sigmoid(z2)    # 5000*25;
    a2 = np.c_[np.ones([m, 1]), a2]     # 5000*26;

    z3 = np.dot(a2, Theta2.T)    # 5000*10;
    a3 = sigmoid(z3)    # 5000*10;           

    y_matrix = np.zeros([m, num_labels]); # 5000*10;              

    for i in range(m):
        if y[i][0]==0:
            y_matrix[i][9] = 1
        else:
            y_matrix[i][y[i][0]-1] = 1     


    sum_t = 0
    for i in range(m):
        for j in range(num_labels):
            sum_t = sum_t - y_matrix[i][j] * np.log(a3[i][j]) - (1-y_matrix[i][j]) * np.log(1-a3[i][j])

    J = sum_t / m

    sum_left = 0
    for i in range(Theta1.shape[0]):
        for j in range(1, Theta1.shape[1]):
            sum_left = sum_left + Theta1[i][j]**2

    sum_right = 0
    for i in range(Theta2.shape[0]):
        for j in range(1, Theta2.shape[1]):
            sum_right = sum_right + Theta2[i][j]**2

    J = J + (lambda_t/(2*m))*(sum_left + sum_right);
    
    return J

In [745]:
def computeGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t):

    Theta1_flat = nn_params[ : hidden_layer_size * (input_layer_size + 1)]
    Theta2_flat = nn_params[hidden_layer_size * (input_layer_size + 1) : ]
    
    Theta1 = np.reshape(Theta1_flat, (hidden_layer_size, input_layer_size+1)) # 25*401;
    Theta2 = np.reshape(Theta2_flat, (num_labels, hidden_layer_size+1)) # 10*26;
    
    # nnCostFunction
    Theta1_grad = np.zeros(Theta1.shape) # 25*401;
    Theta2_grad = np.zeros(Theta2.shape) # 10*26;
    
    m = X.shape[0]

    a1 = np.c_[np.ones([m, 1]), X]    # Add a column of ones to x
    z2 = np.dot(a1, Theta1.T)    # 5000*25;

    a2 = sigmoid(z2)    # 5000*25;
    a2 = np.c_[np.ones([m, 1]), a2]     # 5000*26;

    z3 = np.dot(a2, Theta2.T)    # 5000*10;
    a3 = sigmoid(z3)    # 5000*10;           

    y_matrix = np.zeros([m, num_labels]); # 5000*10;              

    for i in range(m):
        if y[i][0]==0:
            y_matrix[i][9] = 1
        else:
            y_matrix[i][y[i][0]-1] = 1     

    
    ##-----------------------calculate Theta1_grad and Theta2_grad---------------------------
    for i in range(m):
        delta3 = a3[i] - y_matrix[i]
        delta3 = np.reshape(delta3, (1, -1)) # 1*10;
        
        term1 = np.dot(delta3, Theta2)
        term1 = (term1.T)[1:]
        term1 = np.reshape(term1, (1, -1))
        
        term2 = sigmoidGradient(z2[i])
        term2 = np.reshape(term2, (1, -1))
        
        delta2 = np.multiply(term1, term2) # 1*25;

        Theta1_grad = Theta1_grad + np.dot(delta2.T, np.reshape(a1[i], (1, -1)))    # 25*401;
        Theta2_grad = Theta2_grad + np.dot(delta3.T, np.reshape(a2[i], (1, -1)))    # 10*26;    

    Theta1_grad = Theta1_grad / m
    Theta2_grad = Theta2_grad / m
    
    grad = np.concatenate((Theta1_grad.flatten(), Theta2_grad.flatten()))
    
    return grad

In [746]:
def randInitializeWeights(L_in, L_out):
    epsilon_init = 0.12
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    
    return W

In [747]:
input_layer_size  = 400    # 20x20 Input Images of Digits
hidden_layer_size = 25    # 25 hidden units
num_labels = 10          # 10 labels, from 1 to 10   
                          # (note that we have mapped "0" to label 10)

## =========== Part 1: Loading and Visualizing Data =============

X = np.loadtxt('ex4data_x.txt')
y = np.loadtxt('ex4data_y.txt')
y = np.reshape(y, (-1, 1))
y = y.astype(int)
m = X.shape[0]

## ================ Part 2: Loading Parameters ================

Theta1 = np.loadtxt('ex4data_theta1.txt')
Theta2 = np.loadtxt('ex4data_theta2.txt')

print(X.shape)
print(y.shape)
print(Theta1.shape)
print(Theta2.shape)

(5000, 400)
(5000, 1)
(25, 401)
(10, 26)


In [748]:
## ================ Part 3: Compute Cost (Feedforward) ================
print('Feedforward Using Neural Network ...')

lambda_t = 0

nn_params = np.concatenate((Theta1.flatten(), Theta2.flatten()))

J= computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
print('Cost at parameters (loaded from ex4weights): (this value should be about 0.287629)\n', J)

Feedforward Using Neural Network ...
Cost at parameters (loaded from ex4weights): (this value should be about 0.287629)
 0.2876291652410021


In [749]:
## =============== Part 4: Implement Regularization ===============

print('Checking Cost Function (w/ Regularization) ...')

lambda_t = 1

J = computeCost(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
#Gradient = computeGradient(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_t)
#print(Gradient.shape)

print('Cost at parameters (loaded from ex4weights): (this value should be about 0.383770)\n', J)


Checking Cost Function (w/ Regularization) ...
Cost at parameters (loaded from ex4weights): (this value should be about 0.383770)
 0.3837698591466515


In [750]:
## ================ Part 5: Sigmoid Gradient  ================
print('Evaluating sigmoid gradient...')
g = sigmoidGradient(np.array([-1, -0.5, 0, 0.5, 1]))
print('Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:')
print(g)

Evaluating sigmoid gradient...
Sigmoid gradient evaluated at [-1 -0.5 0 0.5 1]:
[0.19661193 0.23500371 0.25       0.23500371 0.19661193]


In [751]:
## ================ Part 6: Initializing Pameters ================
print('Initializing Neural Network Parameters ...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

initial_nn_params = np.concatenate((initial_Theta1.flatten(), initial_Theta2.flatten()))

print(initial_Theta1.shape)
print(initial_Theta2.shape)

Initializing Neural Network Parameters ...
(25, 401)
(10, 26)


In [752]:
## =============== Part 7: Implement Backpropagation ===============
print('Checking Backpropagation... ')
print(debugInitializeWeights(2, 2))

checkNNGradients(0)


Checking Backpropagation... 
[[ 0.0841471   0.09092974  0.014112  ]
 [-0.07568025 -0.09589243 -0.02794155]]


In [753]:
## =============== Part 8: Implement Regularization ===============
print('Checking Backpropagation (w/ Regularization) ... ')


Checking Backpropagation (w/ Regularization) ... 
